In [1]:
# Import the libraries we use.
from bs4 import BeautifulSoup as bsoup
import datetime
import glob
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time
import urllib.request
from urllib.request import urlopen
import zipfile

# Save the initial working directory.
start_directory = os.getcwd()
print(start_directory)

# Create the path to the data directory.
data_directory = os.path.join('.', 'data')
print(data_directory)

C:\Users\Admin\Documents\DataScience\Assignment2
.\data


In [2]:
# Create logfile.
logfile = open("time_data.txt", "w") 
def log_entry(s):
    #print('Date now: %s' % datetime.datetime.now())

    timestamp = '[%s] ' % datetime.datetime.now()
    log_line = timestamp + s + '\n'
    logfile.write(log_line)
    logfile.flush()
    
    # Also write to standard output as a convenience.
    print(log_line)

In [3]:
site = "https://www.lendingclub.com/info/download-data.action"
#print(site)

response = requests.get(site)
response.raise_for_status()
print(response.status_code)

content = response.content
#print(content[0:200])

try:
    response.raise_for_status()
except Exception as exc:
    print('There was a problem: %s' % (exc))

200


In [4]:
base_URL = "https://resources.lendingclub.com"
url = urllib.request.urlopen("https://www.lendingclub.com/info/download-data.action")
content = url.read()
#print(content)

soup = bsoup(content, 'html')
#print(soup) 

#loanStatsFileNamesJS
fileNameDiv = soup.find('div', {"id":"loanStatsFileNamesJS"})
loanFileList = fileNameDiv.text.rstrip("|")
#print(FileList)

# Set the data directory as the current working directory for the downloads.
os.chdir(data_directory)

# Download and extract all the data files.
for fileName in loanFileList.split("|"):
    #print(fileName)
    
    csv_filename = fileName
    if csv_filename.endswith('.zip'):
        csv_filename = csv_filename[:-4]
        
    csv_filepath = os.path.join('.', csv_filename)
    #print(csv_filepath)
    
    # Download the file if it isn't already in our data directory.
    if os.path.exists(csv_filepath):
        print("Already downloaded %s" % csv_filepath)
    else:
        print("Downloading file %s" % csv_filepath)
        file_URL = base_URL + '/' + fileName
        #print(file_URL)
        
        zfile = requests.get(file_URL)
        z = zipfile.ZipFile(io.BytesIO(zfile.content))
        z.extractall()

# Restore the working directory
os.chdir(start_directory)

C:\Users\Admin\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Admin\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
sample_filepath = os.path.join(data_directory, 'LoanStats3a.csv')
data = pd.read_csv(sample_filepath, skiprows=1)
data

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,NaN,NaN,5000.0,5000.0,4975.000000,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.000000,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.000000,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.000000,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.000000,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
5,NaN,NaN,5000.0,5000.0,5000.000000,36 months,7.90%,156.46,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
6,NaN,NaN,7000.0,7000.0,7000.000000,60 months,15.96%,170.08,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
7,NaN,NaN,3000.0,3000.0,3000.000000,36 months,18.64%,109.43,E,E1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
8,NaN,NaN,5600.0,5600.0,5600.000000,60 months,21.28%,152.39,F,F2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
9,NaN,NaN,5375.0,5375.0,5350.000000,60 months,12.69%,121.45,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [6]:
data.shape

(42538, 111)

In [7]:
def concatenate(indir="./", outfilename="./Accepted_LoanData.csv"):
    initial_working_dir = os.getcwd()
    os.chdir(indir)
    
    csvFileList = glob.glob("*.csv")
    dfList = []
   
    # Process the CSV files, without the initial line.
    for csv_filename in csvFileList:
        print(csv_filename)
        
        # Use the file modification time to track when the data was downloaded.
        timestamp = int(os.path.getmtime(csv_filename))
        #print("last modified: %s" % str(timestamp))
        
        df = pd.read_csv(csv_filename, low_memory=False, skiprows=1)
        
        # Add the timestamp into the data.
        df['timestamp'] = timestamp
        print(df.shape)
        
        dfList.append(df)
        
    concatDf = pd.concat(dfList, axis=0, copy=False)
    #concatDf.columns = columns
    concatDf.to_csv(outfilename, index=None)
    print(concatDf.shape)
    
    # Restore the working directory.
    os.chdir(initial_working_dir)

In [8]:
concatenate(indir=data_directory)

LoanStats3a.csv
(42538, 112)
LoanStats3b.csv
(188183, 112)
LoanStats3c.csv
(235631, 112)
LoanStats3d.csv
(421097, 112)
LoanStats_2016Q1.csv
(133889, 112)
LoanStats_2016Q2.csv
(97856, 112)
LoanStats_2016Q3.csv
(99122, 112)
LoanStats_2016Q4.csv
(103548, 112)
(1321864, 112)
